# Data Processing (Betfair)
Covered in this notebook:
1. Process pre-downloaded free Betfair Historical Data (BASIC tier).
2. Produce a cleaned betfair_matches.csv file.

In [ ]:
import os
import betfairutil
import pandas as pd
from string_grouper import match_strings, match_most_similar, \
	group_similar_strings, compute_pairwise_similarities, \
	StringGrouper
from unidecode import unidecode

In [ ]:
%%time
path_to_input_directory = '/Users/tb21144/Library/CloudStorage/OneDrive-UniversityofBristol/SUMMER PROJECT/THESIS/data/BASIC/'
data_in_dict = {'index':['player1','player2','timezone','start_time_utc', 'end_time_utc']}
row_no = 0
for root, dirs, files in os.walk(os.path.abspath(path_to_input_directory)):
    for file in files:
        if not file.startswith('.'):
            #print(os.path.join(root, file))
            pricefile_dict = betfairutil.read_prices_file(os.path.join(root, file),lightweight=True)
            # Go backwards through pricefile market definitionss.
            for n in range(len(pricefile_dict)-1,0,-1):
                if pricefile_dict[n]['marketDefinition']['marketType'] == 'MATCH_ODDS':
                    # Get player names. This can be furthered to get winner/loser.
                    player1 = pricefile_dict[n]['marketDefinition']['runners'][0]['name']
                    player2 = pricefile_dict[n]['marketDefinition']['runners'][1]['name']
                    timezone = pricefile_dict[n]['marketDefinition']['timezone']
                    start_time_utc = pricefile_dict[n]['marketDefinition']['openDate']
                    end_time_utc = pricefile_dict[n]['marketDefinition']['settledTime']
                    row = [player1, player2, timezone, start_time_utc, end_time_utc]
                    row_no += 1
                    data_in_dict[row_no] = row
                    break

In [ ]:
df=pd.DataFrame.from_dict(data_in_dict,orient='index')

In [ ]:
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)

df['start_date'] = df['start_time_utc'].str[:10]
df['start_time'] = df['start_time_utc'].str[11:16]
df['end_date'] = df['end_time_utc'].str[:10]
df['end_time'] = df['end_time_utc'].str[11:16]
df.drop(['start_time_utc'], axis=1, inplace=True)
df.drop(['end_time_utc'], axis=1, inplace=True)

df['start_date'] = pd.to_datetime(df['start_date'])
df.sort_values(by='start_date',inplace=True)
df.reset_index(drop=True, inplace=True)
df.fillna('not_available', inplace=True)

In [ ]:
df.to_csv('betfair_matches.csv')